In [ ]:
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gmean
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA
from typing import List, Set, Tuple, Dict
from river import feature_selection, stream, preprocessing
from tqdm.notebook import tqdm
from enum import Enum, auto

from sklearn.metrics import silhouette_score

import re
import os
import sys
sys.path.append('../')
from vibrodiagnostics import selection, models


PATH_PREFIX = '../../datasets/'
FEATURES_PATH =  os.path.join(PATH_PREFIX, 'features_data')

TD_FD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_AND_FREQ_FEATURES_PATH)
TD_FEATURES = os.path.join(FEATURES_PATH, selection.TIME_FEATURES_PATH)
FD_FEATURES = os.path.join(FEATURES_PATH, selection.FREQ_FEATURES_PATH)

# METRICS_TITLES = ('Correlation', 'F statistic', 'Mutual information')

In [ ]:
faults = {
    'A': {
        'normal': 'normal',
        'imbalance': 'imbalance',
        'horizontal-misalignment': 'misalignment',
        'vertical-misalignment': 'misalignment',
        'underhang-outer_race': 'outer race fault',
        'underhang-cage_fault': 'cage fault',
        'underhang-ball_fault': 'ball fault'
    },
    'B': {
        'normal': 'normal',
        'imbalance': 'imbalance',
        'horizontal-misalignment': 'misalignment',
        'vertical-misalignment': 'misalignment',
        'overhang-cage_fault': 'cage fault',
        'overhang-ball_fault': 'ball fault',
        'overhang-outer_race': 'outer race fault'
    }
}

placements = {
    'A': ['ax', 'ay', 'az'],
    'B': ['bx', 'by', 'bz']
}


domains = {'temporal': TD_FEATURES, 'spectral': FD_FEATURES}
rpm_limit = [False, True]
target = ['fault', 'anomaly_60', 'anomaly_90']
placement = ['A', 'B']
online = [False, True]
GENERATE = False


class ExperimentOutput(Enum):
    COUNTS = auto()
    BEST_SET = auto()
    RANKS = auto()
    SCORES_RANGE = auto()
    PCA = auto()
    SILHOUETTE = auto()
    BEST_CORR = auto()
    BEST_F_STAT = auto()
    BEST_MI = auto()


def get_features_list(domains):
    features = []
    for dname, dataset in domains.items():
        names = pd.read_csv(dataset)
        names = names.columns.str.extract(r'([a-z]{2})_([a-z\_\-]+)')[1].unique()
        features.extend([f'{dname}_{col.strip("_")}' for col in names if not pd.isnull(col)])

    return features


temporal_columns = get_features_list({'temporal': TD_FEATURES})
spectral_columns = get_features_list({'spectral': FD_FEATURES})
all_columns = temporal_columns + spectral_columns

In [ ]:
def silhouette_scores(X_train, X_test, Y_train, Y_test, best_features, pc):
    Y_train = Y_train.reset_index(drop=True).astype('category')
    Y_test = Y_test.reset_index(drop=True).astype('category')

    scaler = MinMaxScaler()
    X_train[X_train.columns] = scaler.fit_transform(X_train)
    X_test[X_test.columns] = scaler.transform(X_test)    

    model = PCA(n_components=pc).fit(X_train)
    X_train_pca = pd.DataFrame(model.transform(X_train))
    X_test_pca = pd.DataFrame(model.transform(X_test))

    return {
        'train': silhouette_score(X_train[best_features], Y_train),
        'test': silhouette_score(X_test[best_features], Y_test),
        'train_pca': silhouette_score(X_train_pca, Y_train),
        'test_pca': silhouette_score(X_test_pca, Y_test)
    }


def pca_explained_variances(X_train: pd.DataFrame, pc: int) -> Dict[str, float]:
    scaler = MinMaxScaler()
    X_train[X_train.columns] = scaler.fit_transform(X_train)
    model = PCA(n_components=pc).fit(X_train)    
    return {f'PC{pc}': var for pc, var in enumerate(model.explained_variance_ratio_, start=1)}


def batch_feature_ranking(X: pd.DataFrame, Y: pd.DataFrame, mode: str = 'rank') -> pd.DataFrame:
    metric_ranks = pd.DataFrame()
    METRICS_OFFLINE = {
        'corr': selection.corr_classif, 
        'f_stat': f_classif,
        'mi': mutual_info_classif
    }

    if mode in METRICS_OFFLINE:
        metric = METRICS_OFFLINE[mode]
        scores = metric(X, Y)
        if isinstance(scores, tuple):
            scores = scores[0]
        leaderboard = (
            pd.DataFrame(zip(X.columns, scores), columns=['feature', 'rank'])
            .set_index('feature')
            .sort_values(by='rank', ascending=False)
        )
        return leaderboard

    elif mode == 'rank':
        for metric_name, metric in METRICS_OFFLINE.items():
            scores = metric(X, Y)
            if isinstance(scores, tuple):
                scores = scores[0]
            leaderboard = (
                pd.DataFrame(zip(X.columns, scores), columns=['feature', 'score'])
                .set_index('feature')
                .sort_values(by='score', ascending=False)
            )
            metric_ranks[metric_name] = leaderboard
        
        ranks = metric_ranks.rank(axis='rows', method='first', ascending=False)
        return ranks.apply(gmean, axis=1).sort_values().to_frame(name='rank')
    

def online_feature_ranking(X: pd.DataFrame, Y: pd.Series, mode: str = 'rank') -> pd.DataFrame:
    METRICS_ONLINE = {
        'corr': selection.Correlation, 
        'f_stat': selection.FisherScore,
        'mi': selection.MutualInformation
    }

    if mode in METRICS_ONLINE:
        metric = METRICS_ONLINE[mode]
        estimator = feature_selection.SelectKBest(similarity=metric(), k=2)
        for xs, ys in stream.iter_pandas(X, Y):
            estimator.learn_one(xs, ys)

        best = [dict(estimator.leaderboard.copy())]
        features = pd.DataFrame.from_records(best)

    elif mode == 'rank':
        estimators = [
            feature_selection.SelectKBest(similarity=metric(), k=2)
            for metric in METRICS_ONLINE.values()
        ]

        best = []
        for xs, ys in stream.iter_pandas(X, Y):
            for method in estimators:
                method.learn_one(xs, ys)

            scores = [method.leaderboard.copy() for method in estimators]
            scores = pd.DataFrame.from_records(scores).T
            ranks = scores.rank(axis='rows', method='first', ascending=False)
            ranks = ranks.apply(gmean, axis=1).to_dict()   # Smallest rank is the best
            best.append(ranks)   

        features = pd.DataFrame.from_records(best)

    return (
        features.tail(1)
        .reset_index(drop=True)
        .T.rename(columns={0: 'rank'})
        .reset_index()
        .rename(columns={'index': 'feature'})
        .set_index('feature')
        .sort_values(by='rank', ascending=True)
    )


def compute_correlations(X: pd.DataFrame, corr_above: float) -> Set[Tuple[str, str]]:
    corr = [
        {'feature_1': k[0], 'feature_2': k[1], 'corr': v}
        for k, v in X.corr().abs().stack().to_dict().items()
        if k[0] != k[1]
    ]
    corr = pd.DataFrame.from_records(corr)

    # Remove correlated features independent of tuple order
    correlations = corr[corr['corr'] >= corr_above][
        ['feature_1', 'feature_2']
    ].to_numpy()
    similar_pairs = set([(a, b) for a, b in correlations])
    similar_pairs.update([(b, a) for a, b in correlations])
    return  similar_pairs


def best_columns(ranks: pd.DataFrame, corr: Set[Tuple[str, str]], n: int) -> List[str]:
    columns = []
    for feature in ranks.index:
        # Make pairs with existing columns
        candidates = [
            col for col in columns 
            if (feature, col) in corr
        ]        
        # Append only if not correlation detected
        if len(candidates) == 0:
            columns.append(feature)

    # Limit to n features
    columns = columns[:n]
    return columns


def best_subset(ranks: pd.DataFrame, corr: Set[Tuple[str, str]], n: int) -> pd.DataFrame:
    columns = best_columns(ranks, corr, n)
    subset = ranks.copy()
    subset['rank'] = False
    subset[subset.index.isin(tuple(columns))] = True
    return subset


def load_source(dataset: str, domain: str, row: dict):
    RPM = 2500
    RPM_RANGE = 500
    features = pd.read_csv(dataset).fillna(0)

    # Choosing rpm range
    if row['rpm_limit']:
        features = features[features['rpm'].between(RPM - RPM_RANGE, RPM + RPM_RANGE, inclusive='both')]

    # Labeling anomaly severity levels
    target = re.search(r'([a-z]+)_?(\d+)?', row['target'])
    anomaly_severity = target.group(2) or '60'
    anomaly_severity = float(anomaly_severity) / 100

    # Choose measurement placement: A or B
    place = row['placement']
    axis = placements[place]
    features = features[features['fault'].isin(tuple(faults[place]))]
    features = models.fault_labeling(features, faults[place], anomaly_severity)

    columns = features.columns.str.startswith(tuple(axis))
    X = features[features.columns[columns]]

    # Select predicted variable column
    label = target.group(1)
    Y = features[label].astype('category')

    # Filter columns in feature domain with window size 2**14
    if domain == 'spectral':
        window_size = 2**14
        X = X.loc[:,X.columns.str.endswith(f'_{window_size}')]
        X.columns = X.columns.str.extract(r'(\w+)_\w+$')[0]

    # Calculate feature magnitudes from 3D vector
    feature_names = get_features_list({domain: dataset})
    result = pd.DataFrame()
    for name in feature_names:              
        # Remove prefix: temporal, spectral
        name = re.search(r'[a-z]+_([\w\_]+)', name).group(1)
        vector_dims = [f'{dim}_{name}' for dim in axis]
        result[name] = X[vector_dims].apply(np.linalg.norm, axis=1)
    X = result

    # Batch / Online hold-out (balance and event sequencing)
    train_size = 0.8
    if row['online']:
        # Shuffle order within severity level and order event with increasing severity
        groups = [
            df.sample(frac=1, random_state=10)
            for i, df in (
                features.sort_values(by='severity_level').groupby('severity_level')
            )
        ]
        rows = list(pd.concat(groups).index)
        X = X.loc[rows].reset_index(drop=True)
        Y = Y.loc[rows].reset_index(drop=True)

        X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y, train_size=train_size, random_state=10
        )   
        X_train, X_test, Y_train, Y_test = (
            X_train.sort_index(), X_test.sort_index(),
            Y_train.sort_index(), Y_test.sort_index()
        )

    else:
        oversample = RandomOverSampler(sampling_strategy='not majority', random_state=10)
        X, Y = oversample.fit_resample(X, Y.to_numpy())
        X.reset_index(drop=True, inplace=True)
        Y = pd.Series(Y)

        X_train, X_test, Y_train, Y_test = train_test_split(
            X, Y, train_size=train_size, stratify=Y, random_state=10
        )

    return X_train, X_test, Y_train, Y_test


def run_experiments(conditions: List[dict], exp_output: ExperimentOutput, pc=3) -> pd.DataFrame:
    experiments = []

    for row in tqdm(conditions):
        experiment = row.copy()

        for domain_label, dataset in domains.items():
            X_train, X_test, Y_train, Y_test = load_source(dataset, domain_label, row)

            # Count samples
            if exp_output == ExperimentOutput.COUNTS:
                experiment.update({'n_train': len(X_train), 'n_test': len(X_test), 'sum': len(X)})
                break

            elif exp_output == ExperimentOutput.PCA:
                experiment = row.copy()
                experiment.update({'domain': domain_label})
                experiment.update(pca_explained_variances(X_train, pc))
                experiments.append(experiment)
                continue

            elif exp_output == ExperimentOutput.SILHOUETTE:
                synonyms = compute_correlations(X_train, corr_above=0.95)
                if row['online']:
                    ranks = online_feature_ranking(X_train, Y_train)
                else:
                    ranks = batch_feature_ranking(X_train, Y_train)
    
                best_features = best_columns(ranks, synonyms, n=3)
                scores = silhouette_scores(X_train, X_test, Y_train, Y_test, best_features, pc)
                experiment = row.copy()
                experiment.update({'domain': domain_label})
                experiment.update(scores)
                experiments.append(experiment)
                continue

            elif exp_output == ExperimentOutput.BEST_SET:
                if row['online']:
                    ranks = online_feature_ranking(X_train, Y_train)
                else:
                    ranks = batch_feature_ranking(X_train, Y_train)
                synonyms = compute_correlations(X_train, corr_above=0.95)
                subset = best_subset(ranks, synonyms, n=3)
                output = subset

            elif exp_output == ExperimentOutput.BEST_CORR:
                if row['online']:
                    ranks = online_feature_ranking(X_train, Y_train, 'corr')
                else:
                    ranks = batch_feature_ranking(X_train, Y_train, 'corr')
                synonyms = compute_correlations(X_train, corr_above=0.95)
                subset = best_subset(ranks, synonyms, n=3)
                output = subset

            elif exp_output == ExperimentOutput.BEST_F_STAT:
                if row['online']:
                    ranks = online_feature_ranking(X_train, Y_train, 'f_stat')
                else:
                    ranks = batch_feature_ranking(X_train, Y_train,'f_stat')
                synonyms = compute_correlations(X_train, corr_above=0.95)
                subset = best_subset(ranks, synonyms, n=3)
                output = subset

            elif exp_output == ExperimentOutput.BEST_MI:
                if row['online']:
                    ranks = online_feature_ranking(X_train, Y_train, 'mi')
                else:
                    ranks = batch_feature_ranking(X_train, Y_train, 'mi')
                synonyms = compute_correlations(X_train, corr_above=0.95)
                subset = best_subset(ranks, synonyms, n=3)
                output = subset
    
            elif exp_output == ExperimentOutput.RANKS:
                if row['online']:
                    ranks = online_feature_ranking(X_train, Y_train)
                else:
                    ranks = batch_feature_ranking(X_train, Y_train)
                output = ranks

            output.reset_index(inplace=True)
            output['feature'] = output['feature'].apply(lambda s: f'{domain_label}_{s}')
            output = dict(zip(list(output['feature']), list(output['rank'])))
            experiment.update(output)

        if exp_output not in (ExperimentOutput.PCA, ExperimentOutput.SILHOUETTE):
            experiments.append(experiment)

    return pd.DataFrame.from_records(experiments)

In [ ]:
column_names = ['rpm_limit', 'target', 'placement', 'online']
initial_conditions = [
    dict(zip(column_names, row)) 
    for row in itertools.product(rpm_limit, target, placement, online)
]

#### Majority voting: feature in subsets

#### 3 member sets

In [ ]:
if GENERATE:
    membership = run_experiments(initial_conditions, ExperimentOutput.BEST_SET)
    membership.to_csv('best_set/rank_product.csv', index=False)
    membership = run_experiments(initial_conditions, ExperimentOutput.BEST_CORR)
    membership.to_csv('best_set/corr.csv', index=False)
    membership = run_experiments(initial_conditions, ExperimentOutput.BEST_F_STAT)
    membership.to_csv('best_set/fstat.csv', index=False)
    membership = run_experiments(initial_conditions, ExperimentOutput.BEST_MI)
    membership.to_csv('best_set/mi.csv', index=False)

In [ ]:
# Globally best features (batch and online)
def globally_best_batch_features(filename):
    best_set_membership = pd.read_csv(filename)

    group = best_set_membership[best_set_membership['online'] == False]
    fig, ax = plt.subplots(1, 2, figsize=(20, 4))
    for i, col in enumerate([temporal_columns, spectral_columns]):
        graph = group[col][group == True].count(axis=0).sort_values(ascending=False)
        print(graph)
        ax[i].grid()
        ax[i].bar([re.search('[a-z]+_(\w+)', s).group(1) for s in graph.index], graph)
    plt.show()

    # Online
    group = best_set_membership[best_set_membership['online'] == True]
    fig, ax = plt.subplots(1, 2, figsize=(20, 4))
    for i, col in enumerate([temporal_columns, spectral_columns]):
        graph = group[col][group == True].count(axis=0).sort_values(ascending=False)
        print(graph)
        ax[i].grid()
        ax[i].bar([re.search('[a-z]+_(\w+)', s).group(1) for s in graph.index], graph)
    plt.show()

In [ ]:
globally_best_batch_features('best_set/rank_product.csv')

In [ ]:
globally_best_batch_features('best_set/corr.csv')

In [ ]:
globally_best_batch_features('best_set/fstat.csv')

In [ ]:
globally_best_batch_features('best_set/mi.csv')

In [ ]:
best_set_membership = pd.read_csv('best_set/rank_product.csv')
agg = pd.DataFrame()
for key, group in best_set_membership.groupby(by=['rpm_limit', 'online']):
    t_situation = group[temporal_columns][group == True].count(axis=0).sort_values(ascending=False).head(3)
    f_situation = group[spectral_columns][group == True].count(axis=0).sort_values(ascending=False).head(3)
    agg[key] = pd.concat([t_situation, f_situation]).index
agg

In [ ]:
best_set_membership = pd.read_csv('best_set/rank_product.csv')
agg = pd.DataFrame()
for key, group in best_set_membership.groupby(by=['rpm_limit', 'online', 'target']):
    t_situation = group[temporal_columns][group == True].count(axis=0).sort_values(ascending=False).head(3)
    f_situation = group[spectral_columns][group == True].count(axis=0).sort_values(ascending=False).head(3)
    agg[key] = pd.concat([t_situation, f_situation]).index
agg

#### Rank product: feature ordering

In [ ]:
if GENERATE:
    best_set_ranks = run_experiments(initial_conditions, ExperimentOutput.RANKS)    # 6 minutes
    best_set_ranks.to_csv('best_set_ranks.csv', index=False)
    best_set_ranks.head()

In [ ]:
best_set_ranks = pd.read_csv('best_set_ranks.csv')
# Globally best features (lower rank is better)

group = best_set_ranks[best_set_ranks['online'] == False]
fig, ax = plt.subplots(1, 2, figsize=(20, 4))
for i, col in enumerate([temporal_columns, spectral_columns]):
    graph = group[col].apply(gmean, axis=0).sort_values(ascending=True)
    print(graph)
    ax[i].grid()
    ax[i].bar([re.search('[a-z]+_(\w+)', s).group(1) for s in graph.index], graph)
plt.show()

# Online
group = best_set_ranks[best_set_ranks['online'] == True]
fig, ax = plt.subplots(1, 2, figsize=(20, 4))
for i, col in enumerate([temporal_columns, spectral_columns]):
    graph = group[col].apply(gmean, axis=0).sort_values(ascending=True)
    print(graph)
    ax[i].grid()
    ax[i].bar([re.search('[a-z]+_(\w+)', s).group(1) for s in graph.index], graph)
plt.show()

In [ ]:
# Summary (absolute counts) - RPM limited/unlimted and machinery element
best_set_ranks = pd.read_csv('best_set_ranks.csv')
agg = pd.DataFrame()
for key, group in best_set_ranks.groupby(by=['rpm_limit', 'online']):
    agg[key] = group[all_columns].apply(gmean, axis=0)
agg

In [ ]:
best_set_ranks = pd.read_csv('best_set_ranks.csv')
agg = pd.DataFrame()
for key, group in best_set_ranks.groupby(by=['rpm_limit', 'online']):
    t_situation = group[temporal_columns].apply(gmean, axis=0).sort_values(ascending=True).head(3)
    f_situation = group[spectral_columns].apply(gmean, axis=0).sort_values(ascending=True).head(3)
    agg[key] = pd.concat([t_situation, f_situation]).index
agg

In [ ]:
# Summary (absolute counts) - RPM limited/unlimted and machinery element
best_set_ranks = pd.read_csv('best_set_ranks.csv')
agg = pd.DataFrame()
for key, group in best_set_ranks.groupby(by=['rpm_limit', 'online', 'target']):
    agg[key] = group[all_columns].apply(gmean, axis=0)
agg

In [ ]:
best_set_ranks = pd.read_csv('best_set_ranks.csv')
agg = pd.DataFrame()
for key, group in best_set_ranks.groupby(by=['rpm_limit', 'online', 'target']):
    t_situation = group[temporal_columns].apply(gmean, axis=0).sort_values(ascending=False).head(3)
    f_situation = group[spectral_columns].apply(gmean, axis=0).sort_values(ascending=False).head(3)
    agg[key] = pd.concat([t_situation, f_situation]).index
agg

### Best features by experiment
- Majority voting
- Ranking

In [ ]:
def extract_feature_names(feature_set):
    return [re.search('[a-z]+_(\w+)', s).group(1) for s in feature_set.index]

def best_featue_set_methods(filename):
    best_set_membership = pd.read_csv(filename)
    feature_sets = []
    indexer = ['placement', 'online', 'rpm_limit', 'target']
    for key, group in best_set_membership.groupby(by=indexer):
        t_situation = group[temporal_columns][group == True].count(axis=0).sort_values(ascending=False).head(3)
        f_situation = group[spectral_columns][group == True].count(axis=0).sort_values(ascending=False).head(3)

        # Extract feature names
        temporal = list(sorted(extract_feature_names(t_situation)))
        spectral = list(sorted(extract_feature_names(f_situation)))

        fset = {'placement': key[0], 'online': key[1], 'rpm_limit': key[2], 'target': key[3], 'temporal': temporal , 'spectral': spectral}
        feature_sets.append(fset)

    return pd.DataFrame.from_records(feature_sets).set_index(indexer)

In [ ]:
best_featue_set_methods('best_set/rank_product.csv')

In [ ]:
best_featue_set_methods('best_set/corr.csv')

In [ ]:
best_featue_set_methods('best_set/fstat.csv')

In [ ]:
best_featue_set_methods('best_set/mi.csv')

In [ ]:
def extract_feature_names(feature_set):
    return [re.search('[a-z]+_(\w+)', s).group(1) for s in feature_set.index]

best_set_membership = pd.read_csv('best_set_ranks.csv')
feature_sets = []
indexer = ['placement', 'online', 'rpm_limit', 'target']
for key, group in best_set_membership.groupby(by=indexer):
    t_situation = group[temporal_columns].apply(gmean, axis=0).sort_values(ascending=False).head(3)
    f_situation = group[spectral_columns].apply(gmean, axis=0).sort_values(ascending=False).head(3)

    # Extract feature names
    temporal = list(sorted(extract_feature_names(t_situation)))
    spectral = list(sorted(extract_feature_names(f_situation)))

    fset = {'placement': key[0], 'online': key[1], 'rpm_limit': key[2], 'target': key[3], 'temporal': temporal , 'spectral': spectral}
    feature_sets.append(fset)

pd.DataFrame.from_records(feature_sets).set_index(indexer)

#### PCA explained variance (batch only)

In [ ]:
column_names = ['rpm_limit', 'target', 'placement', 'online']
batch_initial_conditions = [
    dict(zip(column_names, row)) 
    for row in itertools.product(rpm_limit, target, placement, [False])
]
pca_vars = run_experiments(batch_initial_conditions, ExperimentOutput.PCA)
pca_vars

In [ ]:
def plot_explained_variances(pca_vars_in):
    selected_columns = ['target', 'PC1', 'PC2', 'PC3']
    groupby_columns = ['target']

    pca_A_temporal = pca_vars_in[
        (pca_vars_in['placement'] == 'A') & (pca_vars_in['domain'] == 'temporal')
    ][selected_columns].set_index(groupby_columns)
    pca_B_temporal = pca_vars_in[
        (pca_vars_in['placement'] == 'B') & (pca_vars_in['domain'] == 'temporal')
    ][selected_columns].set_index(groupby_columns)

    pca_A_spectral = pca_vars_in[
        (pca_vars_in['placement'] == 'A') & (pca_vars_in['domain'] == 'spectral')
    ][selected_columns].set_index(groupby_columns)
    pca_B_spectral = pca_vars_in[
        (pca_vars_in['placement'] == 'B') & (pca_vars_in['domain'] == 'spectral')
    ][selected_columns].set_index(groupby_columns)

    fig, ax = plt.subplots(2, 2, figsize=(8, 8))
    pca_A_temporal.plot.bar(stacked=True, grid=True, ax=ax[0][0], title='Temporal features, Placement: A', xlabel='', ylabel='Explained variance')
    pca_B_temporal.plot.bar(stacked=True, grid=True, ax=ax[0][1], title='Temporal features, Placement: B', xlabel='', ylabel='Explained variance')
    pca_A_spectral.plot.bar(stacked=True, grid=True, ax=ax[1][0], title='Spectral features, Placement: A', xlabel='', ylabel='Explained variance')
    pca_B_spectral.plot.bar(stacked=True, grid=True, ax=ax[1][1], title='Spectral features, Placement: B', xlabel='', ylabel='Explained variance')
    plt.tight_layout()
    plt.show()

# No RPM limit
plot_explained_variances(pca_vars[pca_vars['rpm_limit'] == False])

In [ ]:
# RPM limited
plot_explained_variances(pca_vars[pca_vars['rpm_limit'] == True])

#### Baseline for number of features: Principal components explanation power

In [ ]:
# column_names = ['rpm_limit', 'target', 'placement', 'online']
# batch_initial_conditions = [
#     dict(zip(column_names, row)) 
#     for row in itertools.product(rpm_limit, target, placement, [False])
# ]

# pca_components_power = []
# for pc in range(1, 5):
#     e_pc = (
#         run_experiments(batch_initial_conditions, ExperimentOutput.PCA, pc=pc)
#         .drop(columns=['online'])
#         .set_index(['rpm_limit', 'target', 'placement', 'domain'])
#     )
#     e_silh = (
#         run_experiments(batch_initial_conditions, ExperimentOutput.SILHOUETTE, pc=pc)
#         .drop(columns=['online'])
#         .set_index(['rpm_limit', 'target', 'placement', 'domain'])
#     )
#     e = e_pc.join(e_silh)
#     e['n_pc'] = pc
#     pca_components_power.append(e)

In [ ]:
pca_power = pd.concat(pca_components_power).fillna(0).reset_index()
pca_power = pca_power[(pca_power['rpm_limit'] == False) & (pca_power['target'].str.startswith('fault'))]
pca_power = pca_power[['n_pc'] + list(pca_power.columns[pca_power.columns.str.startswith('PC')])]
pca_power.groupby(by='n_pc').mean()

In [ ]:
pca_power = pd.concat(pca_components_power).fillna(0).reset_index()
pca_power = pca_power[(pca_power['rpm_limit'] == False) & (pca_power['target'].str.startswith('anomaly'))]
pca_power = pca_power[['n_pc'] + list(pca_power.columns[pca_power.columns.str.startswith('PC')])]
pca_power.groupby(by='n_pc').mean()

In [ ]:
pca_power = pd.concat(pca_components_power).fillna(0).reset_index()
pca_power = pca_power[(pca_power['rpm_limit'] == False)  & (pca_power['target'].str.startswith('fault'))]
pca_power = pca_power[['n_pc'] + list(pca_power.columns[pca_power.columns.str.startswith('PC')])]
pca_power.groupby(by='n_pc').mean()

In [ ]:
pca_power = pd.concat(pca_components_power).fillna(0).reset_index()
pca_power = pca_power[(pca_power['rpm_limit'] == False) & (pca_power['target'].str.startswith('anomaly'))]
pca_power = pca_power[['n_pc'] + list(pca_power.columns[pca_power.columns.str.startswith('PC')])]
pca_power.groupby(by='n_pc').mean()

In [ ]:
pca_power = pd.concat(pca_components_power).fillna(0).reset_index()
pca_power = pca_power[(pca_power['rpm_limit'] == False) & (pca_power['target'].str.startswith('anomaly'))]
pca_power = pca_power[['n_pc'] + list(pca_power.columns[pca_power.columns.str.endswith('_pca')])]
pca_power.groupby(by='n_pc').mean()

#### Silhouette scores

In [ ]:
scores = run_experiments(batch_initial_conditions, ExperimentOutput.SILHOUETTE)
scores

In [ ]:
def plot_silhouette_scores(scores, cols):
    selected_columns = ['target'] + cols
    groupby_columns = ['target']

    scores_A_temporal = scores[
        (scores['placement'] == 'A') & (scores['domain'] == 'temporal')
    ][selected_columns].set_index(groupby_columns)
    scores_B_temporal = scores[
        (scores['placement'] == 'B') & (scores['domain'] == 'temporal')
    ][selected_columns].set_index(groupby_columns)

    scores_A_spectral = scores[
        (scores['placement'] == 'A') & (scores['domain'] == 'spectral')
    ][selected_columns].set_index(groupby_columns)
    scores_B_spectral = scores[
        (scores['placement'] == 'B') & (scores['domain'] == 'spectral')
    ][selected_columns].set_index(groupby_columns)

    fig, ax = plt.subplots(2, 2, figsize=(10, 10))
    scores_A_temporal.plot.bar(grid=True, ax=ax[0][0], title='Temporal features, Placement: A', xlabel='', ylabel='Silhouette score')
    scores_B_temporal.plot.bar(grid=True, ax=ax[0][1], title='Temporal features, Placement: B', xlabel='', ylabel='Silhouette variance')
    scores_A_spectral.plot.bar(grid=True, ax=ax[1][0], title='Spectral features, Placement: A', xlabel='', ylabel='Silhouette variance')
    scores_B_spectral.plot.bar(grid=True, ax=ax[1][1], title='Spectral features, Placement: B', xlabel='', ylabel='Silhouette variance')
    plt.tight_layout()

plot_silhouette_scores(scores[scores['rpm_limit'] == False], ['train', 'test'])
#plt.suptitle('Best features, all RPM')
plt.show()

In [ ]:
plot_silhouette_scores(scores[scores['rpm_limit'] == False], ['train_pca', 'test_pca'])
plt.suptitle('PCA, all RPM')
plt.show()

#### Dataset sizes

In [ ]:
counters = run_experiments(initial_conditions, ExperimentOutput.COUNTS)  
counters.head()

In [ ]:
samples = (counters[['rpm_limit', 'online', 'target', 'n_train', 'n_test', 'sum']]
.groupby(by=['rpm_limit', 'online', 'target'])
.first())
samples

In [ ]:
samples.loc[:, samples.columns != 'sum'].plot.bar(stacked=True, grid=True)
plt.show()

Feature distribution in diffrent classes

In [ ]:
def boxplot_features(conditions: List[dict]):
    for row in tqdm(conditions):
        experiment = row.copy()

        for domain_label, dataset in domains.items():
            X_train, X_test, Y_train, Y_test = load_source(dataset, domain_label, row)

            # MinMax scaled result
            scaler = MinMaxScaler()
            X_train_scaled = pd.DataFrame()
            X_train_scaled[X_train.columns] = scaler.fit_transform(X_train)

            # Diagonal of covariance matrix to see explained variance cov(A, A) = var(A)
            # Variance threshold
            train_cov = X_train_scaled.cov()
            diagonal_cov = pd.Series(np.diag(train_cov), index=[train_cov.index, train_cov.columns])
            diagonal_cov = diagonal_cov / diagonal_cov.sum()
            diagonal_cov = diagonal_cov.sort_values(ascending=False)
            print(row)
            print(diagonal_cov)

            X_train_scaled['target'] = Y_train
            # Show boxplots split by predicted variable
            X_train_scaled.boxplot(figsize=(15, 5))
            plt.show()
            X_train_scaled.boxplot(figsize=(20, 5), layout=(2, 6), by='target', sharey=False)
            plt.show()

column_names = ['rpm_limit', 'online', 'target', 'placement']
batch_initial_conditions = [
    dict(zip(column_names, row)) 
    for row in itertools.product(rpm_limit, target, placement, [False])
]
boxplot_features(batch_initial_conditions)